<!-- # <span style="color:red">UNDER CONSTRUCTION!!!!</span> -->

# Spoken Language Processing - Instituto Superior Técnico
## Laboratory Assignment 2 - Native Language Identification challenge

# PART 2 - Using pre-trained models


This notebook contains the guide and code cells that permit implementing two advanced systems for native language identification based on:
- speaker representations (utterance-based) obtained with an x-vector model; 
- speech representations (frame-based) obtained with a self-supervised learning (SSL) pre-trained model. 

Besides, the notebook will show how to obtain predictions and score the systems on the development set.

## Before starting

Let's import some modules and make some definitions:

In [ ]:
import os 
import csv 
import time
import pickle
import numpy as np

from pf_tools import CheckThisCell, ETS
from speechbrain.inference.classifiers import EncoderClassifier
from speechbrain.utils.data_utils import split_path
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

LANGUAGES = ('CHI',  'GER',  'HIN',  'ITA')
LANG2ID = {'CHI':1, 'GER':2, 'HIN':3, 'ITA':4}
ID2LANG = dict((LANG2ID[k],k)for k in LANG2ID)

Like in the previous Notebooks, you need to mount Google drive if you are working on Google Colab. Otherwise, you should skip or delete the following code cell:

In [ ]:

raise CheckThisCell ## <---- Remove this torun this cell if you are on Google Colab
from google.colab import drive
drive.mount('/content/drive')


Like in Part1, the audio data is expected to be in a folder with the following format:

```
ets_data/
├── train/
│   └── audio/
│       └──wav files
│   └── key.lst 
│
└── train100/
    └── audio/
        └──wav files
    └── key.lst
... 
```

You must already have this from part 1, so you can set-up your data directory:

In [ ]:

raise CheckThisCell ## <---- Remove this after completing/checking this cell

CWD = os.getcwd()
DATADIR = f'{CWD}/ets_data/' # <--- Change this variable to your working directory containig the ETS data
if not os.path.isdir(DATADIR):
    os.mkdir(DATADIR)
    print(f'WARNING: Your data is not in the folder {DATADIR}')

os.chdir(CWD)
print(f'Your ETS data should be in this folder {DATADIR}')


If you need to download again the data, you can run the following cell:

In [ ]:
raise CheckThisCell

os.chdir(DATADIR)

# download train
!wget http://groups.tecnico.ulisboa.pt/speechproc/pf24/lab2/train.tgz
!tar -xzvf train.tgz

#download train100
!wget http://groups.tecnico.ulisboa.pt/speechproc/pf24/lab2/train100.tgz
!tar -xzvf train100.tgz

#download dev
!wget http://groups.tecnico.ulisboa.pt/speechproc/pf24/lab2/dev.tgz
!tar -xzvf dev.tgz

#download evl
!wget http://groups.tecnico.ulisboa.pt/speechproc/pf24/lab2/evl.tgz
!tar -xzvf evl.tgz

os.chdir(CWD)

## 2.1 Using pre-trained speaker embeddings (x-vectors)

The goal of this part of the lab is to get familiar with and show how to use pre-trained speaker embedings (a.k.a. x-vectors) for speech classification tasks.

There exist plenty of resources and pre-trained models that can be  useful for our task. In particular, x-vectors are the current state of the art approach to obtain speech embeddings that characterize very efficiently speaker or language, among others. X-vectors are neural models typically trained for speaker identification in a supervised way, but also in some cases for other related tasks, like language recognition. Once trained, they can be used to obtain a single embedding vector of fixed dimension for each audio input. This vector corresponds to the activations of one of the layers after the pooling layer. 

The following are examples of x-vector models available in the `speechbrain` module: 

- `speechbrain/spkrec-ecapa-voxceleb`: trained using a large speaker corpus for speaker verification: https://huggingface.co/speechbrain/spkrec-ecapa-voxceleb

- `speechbrain/lang-id-voxlingua107-ecapa`: trained using a large corpus of 107 languages for language identification: https://huggingface.co/speechbrain/lang-id-voxlingua107-ecapa

- `speechbrain/lang-id-commonlanguage_ecapa`: trained using a large corpus of 45 languages for language identification: https://huggingface.co/speechbrain/lang-id-commonlanguage_ecapa


The following code cell shows how to import one of those models to obtain an embedding vector:

In [ ]:

xvector_model = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir=f"{CWD}/tmp")

signal = xvector_model.load_audio(f'{DATADIR}/train100/audio/train_0005.wav')
emb =  xvector_model.encode_batch(signal)


These (very informative) embedding vectors can be used to train simple models for several speech classification tasks, achieving excellent results. In particular, in this lab assignment, we will train a simple linear SVM on top of these x-vectors. 

### 2.1.1 Extracting x-vectors for the ETS datasets

Just like in Part1, we will code the feature transformation to process all data and obtain x-vectors. In this case, the function should receive as arguments the audio filename and an instance of `EncoderClassifier` (the x-vector model) and return the numpy array with the features. You must complete the following code using the previous example:

In [ ]:

raise CheckThisCell ## <---- Remove this after completing/checking this cell

# This function receives a filename and one encoder model 
# (for instance, xvector_model object in previous cell example) and returns 
# the extracted x-vectors as a numpy array of shape (1, embedding_dimension).
# Notice that the encoder extractor module generates a softlink to each audio 
# file. To avoid generating "junk", you can delete this link also in the
# function 

def extract_xvec(filename, emb_model):
    # LABWORK :: CODE TO INSERT HERE
    embedding = None 
    # LABWORK :: CODE TO INSERT HERE
    
    # Remove the "annoying" soft-link
    _, fl = split_path(filename)
    if os.path.islink(fl):
        os.remove(fl)
    return embedding

# This must return a numpy array of shape (1,256). 
# The 256 may depend on the specific model
emb = extract_xvec(f'{DATADIR}/train100/audio/train_0005.wav' , xvector_model)

# Check that this shape and type
emb.shape, type(emb)

Let's generate the x-vectors for all our data sets using the ETS class and store in disk. Like in Part 1, we can keep different transformation configurations in a dictionary for later usage. In this assignment, you can try to different x-vector models.

In [ ]:

raise CheckThisCell ## <---- Remove this after completing/checking this cell

transform = { 
                'spkrec-ecapa-voxceleb' : # <--- You chan look for different models in speechbrain
                {
                    'audio_transform': 
                        lambda x : extract_xvec(x, 
                            emb_model = EncoderClassifier.from_hparams(
                                        source="speechbrain/spkrec-ecapa-voxceleb", # <--- You chan look for different models in speechbrain
                                        savedir=f"{CWD}/tmp/spkrec-ecapa-voxceleb"
                                        )
                            ), ## <--- You need to modify this here
                    'chunk_transform': None,
                    'chunk_size': 0,
                    'chunk_hop':0   
                }
            }

transform['lang-id-voxlingua107-ecapa'] = {
                    'audio_transform': 
                        lambda x : extract_xvec(x, 
                            emb_model = EncoderClassifier.from_hparams(
                                        source="speechbrain/lang-id-voxlingua107-ecapa", # <--- You chan look for different models in speechbrain
                                        savedir=f"{CWD}/tmp/lang-id-voxlingua107-ecapa"
                                        )
                            ), ## <--- You need to modify this here
                    'chunk_transform': None,
                    'chunk_size': 0,
                    'chunk_hop':0   
                }

# Download and feature extract
trainset = 'train100'
transform_id = 'spkrec-ecapa-voxceleb' 

train_ets = ETS(DATADIR, trainset, 
                 transform_id=transform_id, 
                 audio_transform=transform[transform_id]['audio_transform'], 
                 chunk_transform=transform[transform_id]['chunk_transform'],
                 chunk_size=transform[transform_id]['chunk_size'], 
                 chunk_hop=transform[transform_id]['chunk_hop']
                )

dev_ets = ETS(DATADIR, 'dev', 
                 transform_id=transform_id, 
                 audio_transform=transform[transform_id]['audio_transform'], 
                 chunk_transform=transform[transform_id]['chunk_transform'],
                 chunk_size=transform[transform_id]['chunk_size'], 
                 chunk_hop=transform[transform_id]['chunk_hop']
                )

evl_ets = ETS(DATADIR, 'evl', 
                 transform_id=transform_id, 
                 audio_transform=transform[transform_id]['audio_transform'], 
                 chunk_transform=transform[transform_id]['chunk_transform'],
                 chunk_size=transform[transform_id]['chunk_size'], 
                 chunk_hop=transform[transform_id]['chunk_hop']
                )

### 2.1.2 Training the SVM model 

As mentioned before, our native language identification system will be a simple SVM model (4-classes) using x-vectors as features. Similarly to Part 1, we must iterate the ETS data instances to store the x-vectors and the labels in numpy arrays:

In [ ]:
# Place train data and reference in a single array train_data and train_label  
start = time.time()
train_data = []
train_labels = []
for data, label, basename in train_ets:
        train_data.append(data)
        train_labels.append(np.full(data.shape[0], label)) 

train_data = np.concatenate(train_data)
train_labels = np.concatenate(train_labels)

print(f'Finished reading all data in {time.time() - start}')

# Place dev data and reference in a single array dev_data and dev_label
start = time.time()
dev_data = []
dev_labels = []
dev_filenames = []
for data, label, basename in dev_ets:
        dev_data.append(data)
        dev_labels.append(np.full(data.shape[0], label)) 
        dev_filenames.append(basename)

dev_data = np.concatenate(dev_data)
dev_labels = np.concatenate(dev_labels)

print(f'Finished reading all data in {time.time() - start}')

# Place evl data and reference in a single array evl_data
start = time.time()
evl_data = []
evl_filenames = []
for data, label, basename in evl_ets:
        evl_data.append(data)
        evl_filenames.append(basename)

evl_data = np.concatenate(evl_data)

print(f'Finished reading all data in {time.time() - start}')

Now, we will use `sklearn` linear SVM to train our classifier and save it for later use.

In [ ]:
model = LinearSVC() ### <---- a linear SVM
model.fit(train_data, train_labels)  ## <---- train MODEL

model_path = f'{DATADIR}/{trainset}/models/'
if not os.path.isdir(model_path):
    os.mkdir(model_path)
    
model_name = f'{model_path}/svm_{transform_id}'
os.mkdir(f'{model_name}')

pickle.dump(model, open(f'{model_name}/model.pkl', 'wb'))

It should be extremely easy to experiment other models provided in the `sklearn` module, including SVMs with other kernels, Random Forests, etc.


### 2.1.3 Prediction using the x-vector based model

Finally, let's predict on both the dev and evl test sets:

In [ ]:
# LOAD THE RIGHT MODEL IF YOU NEED TO:
# model_name = f'{DATADIR}/{trainset}/models//svm_{transform_id}'
# model = pickle.load(open(f'{model_name}/model.pkl', 'rb'))

dev_results = model.predict(dev_data) #  Predict dev
evl_results = model.predict(evl_data) #  Predict test

We can check the performance on the dev set:

In [ ]:
ref, hyp = dev_labels, dev_results
print(classification_report(ref, hyp, target_names=LANGUAGES))
print(accuracy_score(ref, hyp))

In [ ]:
conf_matrix = confusion_matrix(ref, hyp)

fig, ax = plt.subplots(figsize=(3.0, 3.0))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center')

plt.xlabel('Predictions', fontsize=12)
plt.ylabel('Actuals', fontsize=12)
plt.title(f'Confusion Matrix\n(Accuracy {100*accuracy_score(ref, hyp):.2f})', fontsize=12)
plt.show()

If things went as expected, you should around 85% accuracy result!!

At this point, you can explore different x-vector model configurations for feature extraction and alternative models to the linear SVM. Notice that it may be difficult to improve this strong system. Finally, you can generate the final prediction file and make a submission to the  [Kaggle competition](https://www.kaggle.com/t/312cd4200cfb4e138ea9372ce5bc33fd):

In [ ]:
# trainset = 'train100' ## <--- CHANGE THIS ACCORDINGLY
# transform_id = 'spkrec-ecapa-voxceleb' ## <--- CHANGE THIS ACCORDINGLY

group, system = '00', f'svm_{trainset}_{transform_id}'
with open(f'{CWD}/g{group}_{system}.csv', 'w') as file:
    csv_writer = csv.writer(file) # CSV writer
    csv_writer.writerow(('fileId', 'Lang')) # Header of the CSV

    # Save dev results
    for lang, file in zip(dev_results, dev_filenames):
        csv_writer.writerow((file, LANG2ID[lang]))
    # Save evl results
    for lang, file in zip(evl_results, evl_filenames):
        csv_writer.writerow((file, LANG2ID[lang]))
        

## 2.2 Using self-supervised pre-trained models

The goal of this part of the laboratory is to expose students to modern tools and methods for speech classification.
In particular, we will use the [s3prl](https://github.com/s3prl/s3prl) toolkit to build a native language identification system based on self-supervised learning (SSL) models as feature extraction.

[s3prl](https://github.com/s3prl/s3prl) is an open source toolkit, which stands for Self-Supervised Speech Pre-training and Representation Learning. Self-supervised speech pre-trained models are called upstream in this toolkit and are utilized in various downstream tasks.

The toolkit permits pre-training upstream models, load already pre-trained upstream models and/or utilize these upstream models in lots of downstream tasks already defined.

For this lab, the faculty team configured a downstream task and a simple model specifically for our native language identification task and data. The model consists of a projection layer, followed by an average pooling, and a linear output layer.

In this part of the lab, students are  expected to *play* with the different upstream models to build the best possible native language identification system. 
In particular, students are encouraged to explore and discover which of the available SSL models can be a better candidate for their classification system. Note that using a large SSL model will make the training process very slow. So, you must choose wisely depending for instance on the reported performance in similar tasks.

Besides playing with the different upstream models, interested students can try to modify some of the details of the "expert" downstream model. This can be done following some of the many examples already included in the toolkit as a starting point, but it may be challenging. 

**In contrast to the x-vector based system and the notebook of Part 1, for the SSL system, we will make use of some Python scripts that are part of the S3PRL framework. These are typically run in a terminal, so, some of the following steps may be simpler to run in a terminal, rather than as cell in the Notebook itself.**

### 2.2.1 Installing the S3PRL toolkit
Let's start by cloning the repository of the `s3prl` toolkit:

In [ ]:
!git clone https://github.com/s3prl/s3prl.git

A new folder named  `s3prl/` with the contents of the toolkit has been created. We'll now install the toolkit itself.

In [ ]:
S3PRLDIR = CWD + '/s3prl/'
os.chdir(S3PRLDIR)
!pip install -e .

### 2.2.2 Configuring the downstream task
Let's create the downstream native language identification task. In the `s3prl/downstream/` there are plenty of examples. The faculty team took one of those as an example to create the configuration needed for this lab assignment. Let's download and copy it to the downstream folder: 

In [ ]:
os.chdir(f'{S3PRLDIR}/s3prl/downstream/') #  change to the downstream folder
!wget http://groups.tecnico.ulisboa.pt/speechproc/pf24/lab2/nli_s3prl_downstream.tgz # <--- download the lab specific downstream task
!tar -xzvf nli_s3prl_downstream.tgz  #  unzip
!rm nli_s3prl_downstream.tgz
os.chdir(S3PRLDIR)

Have a look to the contents of the folder `s3prl/downstream/native_language_identification/`. There are some important files that help to define the task:
- `dataset.py`: this file provides the class that permits loading the ETS data. Something similar to the ETS class used in previously, but following the formatting rules of the s3prl toolkit. **You don't need to change anything here**.
- `expert.py`: this file defines the expert downstream task. In this case, the expert takes the output of the upstream model (configurable), applies a projection layer, and then a classification model (configurable) to obtain the final predictions. **You don't need to change anything here**.
- `model.py`: this file contains the definitions of the model after the projection. We could include several configurations that can later be selected when we run the actual experiment. The model included is just an average pooling (that reduces the time dimension to a single vector) followed by a linear output layer. **You don't need to change anything here, but you may want to explore other configurations following the examples of other downstream tasks included in s3prl**.
- `config.yaml`: this file permits configuring some parameters of your experiment, including the path that contains the task data and the training set that is going to be used (either train or train100). 

Let's configure our experiment: 

Edit the 'file_path' entry in the configuration file `downstream/native_language_identification/config.yaml` to the folder containing the data:

```yaml
downstream_expert:
    datarc:
        file_path: "your_path/ets_data"
```



And also edit the config file, to use either the "train100" partition or the total training data "train"  by just editing the following entry of the `downstream/native_language_identification/config.yaml`:

```yaml
downstream_expert:
    datarc:
        ...
        train: "train100"
```

You may also want to reduce the number of training steps to 1000 or 2000 for quick experimentation of different configurations:

```yaml
runner:
  total_steps: 5000
  ...
```


### 2.2.3 Training the downstream model
Now it's time for training. For that, we will use the Pythons script `run_downstream.py` in train mode and in which we will set:
- an arbitrary name for identifying this experiment, in which the results will be saved ( `ExpName`)
- the upstream model to be used, for instance, `fbank`. You can check for more SSL pretrained available models  here https://s3prl.github.io/s3prl/tutorial/upstream_collection.html
- the downstream task, in this case "native_language_identification"


```bash
python3 run_downstream.py -n ExpName -m train -u fbank -d native_language_identification
```

Since this can take a while (actually, a lot depending on the chosen upstream model and your  computational resources), you probably want to run this in a terminal, rather than inside the Notebook:


In [ ]:
os.chdir(f'{S3PRLDIR}/s3prl')
!python3 run_downstream.py -n fbank -m train -u fbank -d native_language_identification


HINT: if the training process is interrupted for some reason, you can continue from the last saved checkpoint (a checkpoint is saved every 200 iterations):

```bash
python3 run_downstream.py -m train -e result/downstream/fbank/states-XXXX.ckpt
```

### 2.2.4 Classification and performance of the dev set

The training process created a folder containing some training results `result/downstream/{ExpName}`:

In [ ]:

os.chdir(f'{S3PRLDIR}/s3prl')
!ls result/downstream/fbank/

Some interesting files: 
- `dev_predict.txt` contains the predictions on the dev set, 
- `dev-best.ckpt` contains the model parameters for the best checkpoint,
- `log.log` contains information of the training process, including the identification accuracy in the train and dev sets. 

Let's load the prediction and reference files to compute the performance of our system:

In [ ]:
hyp = [line.strip().split()[-1] for line in open(f'result/downstream/fbank/dev_predict.txt')]
ref = [line.strip().split()[-1] for line in open(f'result/downstream/fbank/dev_truth.txt')]

print(classification_report(ref, hyp, target_names=LANGUAGES))
print(accuracy_score(ref, hyp))

That was disappointing!! The `fbank` is **a very bad upstream model!!!**. 

The goal of this lab is for you to get familiar with and experiment with some of the most popular speech SSL models. You can start with those commented in the theoretical lessons or the ones that have shown good performance in similar tasks. Be careful (and wise) in your decisions: experiments may be slow! 

### 2.2.5 Classification of the evl set 

Now that we already trained our downstream model, we can use it to predict  the blind evl set. For this purpose, we will use again the `run_downstream.py` script in evaluate mode and we need to select the actual model to use:


```bash
python3 run_downstream.py -m evaluate -e result/downstream/fbank/dev-best.ckpt
```

NOTE: Ignore the test accuracy reported at the end (we  don't have the groundtruth).

In [ ]:
!python3 run_downstream.py -m evaluate -e result/downstream/fbank/dev-best.ckpt

The `test_predict.txt` file contains the predictions of this model for the evl set.

### 2.2.6 Create the final predictions file and submit to the challenge

Like in Part 1, we will create the predictions file in the expected format:

In [ ]:

trainset = 'train100'
upstream_id = 'fbank' ## <--- CHANGE THIS ACCORDINGLY
group, system = '00', f'ssl_{trainset}_{upstream_id}'

filename_dev = f'{S3PRLDIR}/s3prl/result/downstream/{upstream_id}/dev_predict.txt'
filename_evl = f'{S3PRLDIR}/s3prl/result/downstream/{upstream_id}/test_predict.txt'


with open(f'{CWD}/g{group}_{system}.csv', 'w') as file:
    
    csv_writer = csv.writer(file) # CSV writer
    csv_writer.writerow(('fileId', 'Lang')) # Header of the CSV

    results_dev = [l.strip().split() for l in open(filename_dev, 'r')]
    results_evl = [l.strip().split() for l in open(filename_evl, 'r')]

    # Save dev results
    for file_id, lang in results_dev:
        file_id = file_id.split('-')[-1]
        lang = LANG2ID[lang]
        csv_writer.writerow((file_id, lang))
        
    # Save evl results
    for file_id, lang in results_evl:
        file_id = file_id.split('-')[-1]
        lang = LANG2ID[lang]
        csv_writer.writerow((file_id, lang))

Finally, you can submit your prediction(s) in the  [Kaggle competition](https://www.kaggle.com/t/312cd4200cfb4e138ea9372ce5bc33fd).



# Contacts and support
You can contact the professors during the classes or the office hours.

Particularly, for this second laboratory assignment, you should contact Prof. Alberto Abad: alberto.abad@tecnico.ulisboa.pt


